In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json 
import os
import numpy as np
import tensorflow as tf
import tqdm
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

In [4]:
# Importing data
df = pd.read_csv('open/train.csv', parse_dates=['일자'], index_col='일자')
df.head()

,종목코드,종목명,거래량,시가,고가,저가,종가
일자,,,,,,,
2021-06-01,A060310,3S,166690,2890,2970,2885,2920
2021-06-01,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2021-06-01,A006840,AK홀딩스,103691,35500,35600,34150,34400
2021-06-01,A054620,APS,462544,14600,14950,13800,14950
2021-06-01,A265520,AP시스템,131987,29150,29150,28800,29050


In [5]:
# Make a list of companies
companies = list(set(df['종목코드'].values))
# From DACON
companies_list = df['종목코드'].unique()

In [6]:
df_close = df['종가']

In [7]:
# Stationary Point: static mean & variance
# Reference: https://www.kaggle.com/code/thibauthurson/stock-price-prediction-with-lstm-multi-step-lstm
def test_stationarity(timeseries):    
    # rolling window calculations of 15 days
    rolmean = timeseries.rolling(15).mean()
    rolstd = timeseries.rolling(15).std()

    #Perform Dickey-Fuller test provided by statsmodels
    dftest = adfuller(timeseries, autolag='AIC') # [Test Statistic, p-value, Lag Used, Number of Observation Used]
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value',\
                                             '#Lags Used','Number of Observations Used'])
    p_value = dftest[1]
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value

    return p_value, dfoutput

In [8]:
def to_stationary(timeseries):
   df_close = timeseries

   # To deal with trend and seasonality: differentiating
   df_close_log = df_close.apply(np.log)
   df_close_tf = df_close_log.apply(np.sqrt)

   # Data becomes stationary series with p-value less than 5% threshold -> null hypothesis rejected
   df_close_shift = df_close_tf - df_close_tf.shift()
   df_close_shift.dropna(inplace=True) # remove NA values
   
   p_value_tf, dfstats_tf = test_stationarity(df_close_shift)
   if p_value_tf < 0.05:
      return df_close
   return df_close_shift

# Esemble Model

In [9]:
# Importing the Keras libraries and packages
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

def lstm_model(sliding_batch):
   tf.random.set_seed(40)
   model= Sequential()

   model.add(LSTM(units=200,return_sequences=True, input_shape=(sliding_batch, 1))) # univariate 
   model.add(Dropout(rate=0.2))

   model.add(LSTM(units=200,return_sequences=True))
   model.add(Dropout(rate=0.2))
   
   model.add(LSTM(units=200,return_sequences=True))
   model.add(Dropout(rate=0.2))

   model.add(LSTM(units=200,return_sequences=True))
   model.add(Dropout(rate=0.2))
   
   model.add(LSTM(units=200,return_sequences=True))
   model.add(Dropout(rate=0.2))

   model.add(LSTM(units=200))
   model.add(Dropout(rate=0.2))

   # Adding the Output Layer
   model.add(Dense(units=1))

   # Compiling the Model
   # Because we're doing regression hence mean_squared_error
   model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.05), metrics=[tf.keras.metrics.MeanAbsoluteError()])
   
   return model

In [10]:
def data_scaler(df_shift, test_days, sliding_batch):
    total_data = len(df_shift) #494
    training_days = total_data - test_days # Use all data excluding test_days for training data
    df_shift = df_shift.values.reshape(-1,1)
    train_data = df_shift[0:training_days] # until training_days
    test_data = df_shift[training_days:] # all other data

    scaler = MinMaxScaler(feature_range=(0,1))
    scaler = scaler.fit(train_data)
    
    # Transform adaptive to training data
    train_scaled_data = scaler.transform(train_data) # just put values  (394,1)
    test_scaled_data = scaler.transform(test_data) # (100,1)
    
    X_train = []
    y_train = []
    # Sliding Window to store 15 days per batch
    for i in range(sliding_batch, training_days):
        X_train.append(train_data[i-sliding_batch:i, 0])
        y_train.append(train_data[i,0])
    X_train, y_train = np.array(X_train), np.array(y_train) # (379, 15) - sliding one day with 15 days as a batch
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)) 
    
    X_test = []
    y_test = test_data[sliding_batch:] # a batch of data labeled as stock price of the next day
    for i in range(sliding_batch, test_days):
        X_test.append(test_scaled_data[i-sliding_batch:i, 0]) #(85,15) - sliding one day with 15 days as a batch
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1)) #(129,60,1)
    
    return X_train, y_train, X_test, y_test, scaler

In [11]:
# Import modules
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Parameters for dataset configuration
sliding_batch = 15
test_days = 100 
epoch = 15
batch_sizes = 15

# For one company
# Input: a company, training days (how many days to train), n_features, n_steps_out (sliding window), test_days (how many days to test on?)
# Output: a list of predictions per each day (size: (test_days, 1))
def esemble(df_shift, test_days, sliding_batch, epoch, batch_sizes):

   # df_shift: dataframe after stationary test
   # Training Set & Testing Set
   X_train, y_train, X_test, y_test, scaler = data_scaler(df_shift, test_days, sliding_batch)
   
   # For putting data into XGBoost
   X_train_xg, y_train_xg, X_test_xg, y_test_xg = X_train.squeeze(), y_train.squeeze(), X_test.squeeze(), y_test.squeeze()
   
   # Model setting
   model1 = lstm_model(sliding_batch)
   model2 = XGBRegressor(objective='reg:squarederror', n_estimators=1000, random_state = 40)
   
   # Fitting the first model
   model1.fit(X_train, y_train, epochs=epoch, batch_size = batch_sizes)
   
   # Prediction & evaluation from the first model 
   results = model1.evaluate(X_test, y_test, batch_size=32) #[MSE, MAE]
   y_pred_1 = model1.predict(X_test) #(85, 1)
   
   # Fitting for the second model
   model2.fit(X_train_xg, y_train_xg)
   y_pred_2 = model2.predict(X_test_xg) #(85,)
   y_pred_2 = y_pred_2.reshape(-1,1)
   xg_mae = mean_absolute_error(y_test_xg, y_pred_2)

   # Esemble
   lstm_mae = results[1] # Get MAE from evaluation of lstm model
   sum_mae = lstm_mae + xg_mae
   avg_mae = (sum_mae)/2
   # Can adapt the weight by different functions
   y_pred = y_pred_1 * (1-lstm_mae/sum_mae) + y_pred_2 * (1-xg_mae/sum_mae) #(85, 85)
   
   # For bringing it back to price
   train_original = df_shift.iloc[:-test_days]
   test_original = df_shift.iloc[-test_days:]
   
   return y_pred_1, y_pred_2, y_pred

In [15]:
def to_price(df_original, df_shift, p_value, predictions, test_days, sliding_batch):
    total_data = len(df_close) #494
    training_days = total_data - test_days # Use all data excluding test_days for training data
    # Shift data: 493 as it has first day as a reference
    test_original = df_original.iloc[training_days:]
    test_original = test_original[sliding_batch:] 
    pred_data = pd.DataFrame(predictions[:,0], test_original.index,columns=['Close'])
    if p_value > 0.05:
        pred_data = pred_data.apply(np.square)
        y_pred = pred_data.apply(np.exp)
        pred_data['Close'] = y_pred['Close'] + df_original.shift().values[(training_days+sliding_batch):]
    return pred_data

In [18]:
# Parameters for dataset configuration
sliding_batch = 15
test_days = 100 
epoch = 15
batch_sizes = 15


# For all the companies
# Input: list_companies (a list of companies), data_df (a dataframe)
def all_companies(list_companies, data_df, feature: str):
   results_df = pd.DataFrame(columns=['종목코드', 'final_return'])
   results_df_lstm = pd.DataFrame(columns=['종목코드', 'final_return'])
   results_df_xg = pd.DataFrame(columns=['종목코드', 'final_return'])
   for code in tqdm.tqdm(list_companies):
      df = data_df[data_df['종목코드'] == code]
      df_feature = df[feature]
      
      # function for test-stationary & shift it to appropriate format
      p_value, dfstats = test_stationarity(df_feature)
      # If null hypothesis not rejected, shift the data so that it can be stationary
      if p_value > 0.05:
         df_feature_shift = to_stationary(df_feature)
      # p_value_shift, dfstats_shift = test_stationarity(df_feature_shift) # not necessary for now
      
      # Training Data & Test Data
      X_train, y_train, X_test, y_test, scaler = data_scaler(df_feature_shift, 100, 15)
      y_1, y_2, predictions = esemble(df_feature_shift, test_days, sliding_batch, epoch, batch_sizes)
      
      prices = to_price(df_close, df_feature_shift, p_value, predictions, test_days, sliding_batch) #esemble
      prices_lstm = to_price(df_close, df_feature_shift, p_value, y_1, test_days, sliding_batch)
      prices_xg = to_price(df_close, df_feature_shift, p_value, y_2, test_days, sliding_batch)
      
      # 최종 수익률 계산
      final_return = (prices.iloc[-1] - prices.iloc[0]) / prices.iloc[0]
      final_return_lstm = (prices_lstm.iloc[-1] - prices_lstm.iloc[0]) / prices_lstm.iloc[0]
      final_return_xg = (prices_xg.iloc[-1] - prices_xg.iloc[0]) / prices_xg.iloc[0]
      
      # 결과 저장
      results_df = pd.concat([results_df, pd.DataFrame({'종목코드': [code], 'final_return': [final_return]})])
      results_df_lstm = pd.concat([final_return_lstm, pd.DataFrame({'종목코드': [code], 'final_return': [final_return_lstm]})])
      results_df_xg = pd.concat([final_return_xg, pd.DataFrame({'종목코드': [code], 'final_return': [final_return_xg]})])
      #results_df = results_df.concat([results_df, {'종목코드': code, 'final_return': final_return}], ignore_index=True)
   results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
   results_df_lstm['순위'] = results_df_lstm['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
   results_df_xg['순위'] = results_df_xg['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
   return results_df, results_df_lstm, results_df_xg

In [19]:
data, data_lstm, data_xg = all_companies(companies_list, df, '종가')

  0%|          | 0/2000 [00:00<?, ?it/s]

Epoch 1/15
26/26 [==============================] - 34s 190ms/step - loss: 9653890.0000 - mean_absolute_error: 3065.8152
Epoch 2/15
26/26 [==============================] - 5s 190ms/step - loss: 8768136.0000 - mean_absolute_error: 2918.0339
Epoch 3/15
26/26 [==============================] - 5s 184ms/step - loss: 7936651.0000 - mean_absolute_error: 2771.4641
Epoch 4/15
26/26 [==============================] - 4s 168ms/step - loss: 7187498.5000 - mean_absolute_error: 2632.3118
Epoch 5/15
26/26 [==============================] - 5s 177ms/step - loss: 6484908.0000 - mean_absolute_error: 2496.0554
Epoch 6/15
26/26 [==============================] - 5s 177ms/step - loss: 5834009.0000 - mean_absolute_error: 2361.9893
Epoch 7/15
26/26 [==============================] - 5s 175ms/step - loss: 5276484.0000 - mean_absolute_error: 2240.7075
Epoch 8/15
26/26 [==============================] - 4s 165ms/step - loss: 4719754.0000 - mean_absolute_error: 2113.7869
Epoch 9/15
26/26 [=====================

  0%|          | 1/2000 [01:54<63:49:28, 114.94s/it]

Epoch 1/15
26/26 [==============================] - 34s 122ms/step - loss: 38244772.0000 - mean_absolute_error: 6136.9316
Epoch 2/15
26/26 [==============================] - 3s 127ms/step - loss: 36480252.0000 - mean_absolute_error: 5991.2837
Epoch 3/15
26/26 [==============================] - 4s 119ms/step - loss: 34760928.0000 - mean_absolute_error: 5846.4536
Epoch 4/15
26/26 [==============================] - 3s 114ms/step - loss: 33129246.0000 - mean_absolute_error: 5705.3560
Epoch 5/15
26/26 [==============================] - 3s 116ms/step - loss: 31532130.0000 - mean_absolute_error: 5562.9155
Epoch 6/15
26/26 [==============================] - 3s 117ms/step - loss: 30007734.0000 - mean_absolute_error: 5424.3340
Epoch 7/15
26/26 [==============================] - 3s 112ms/step - loss: 28580712.0000 - mean_absolute_error: 5292.0098
Epoch 8/15
26/26 [==============================] - 3s 113ms/step - loss: 27153748.0000 - mean_absolute_error: 5154.4507
Epoch 9/15
26/26 [=============

  0%|          | 2/2000 [03:31<57:39:23, 103.89s/it]

...

Epoch 1/15
26/26 [==============================] - 13s 61ms/step - loss: 28752934.0000 - mean_absolute_error: 5261.0225
Epoch 2/15
26/26 [==============================] - 1s 56ms/step - loss: 27307740.0000 - mean_absolute_error: 5121.7495
Epoch 3/15
26/26 [==============================] - 2s 61ms/step - loss: 25910386.0000 - mean_absolute_error: 4983.3252
Epoch 4/15
26/26 [==============================] - 2s 58ms/step - loss: 24586386.0000 - mean_absolute_error: 4847.5415
Epoch 5/15
26/26 [==============================] - 2s 58ms/step - loss: 23307254.0000 - mean_absolute_error: 4715.5493
Epoch 6/15
26/26 [==============================] - 2s 65ms/step - loss: 22056408.0000 - mean_absolute_error: 4580.6060
Epoch 7/15
26/26 [==============================] - 1s 56ms/step - loss: 20917276.0000 - mean_absolute_error: 4454.3057
Epoch 8/15
26/26 [==============================] - 2s 60ms/step - loss: 19780982.0000 - mean_absolute_error: 4324.5151
Epoch 9/15
26/26 [=====================

  2%|▏         | 39/2000 [45:32<23:12:55, 42.62s/it]

Epoch 1/15
26/26 [==============================] - 13s 51ms/step - loss: 25988052.0000 - mean_absolute_error: 5041.9355
Epoch 2/15
26/26 [==============================] - 1s 51ms/step - loss: 24588782.0000 - mean_absolute_error: 4901.5776
Epoch 3/15
26/26 [==============================] - 1s 52ms/step - loss: 23223264.0000 - mean_absolute_error: 4759.9824
Epoch 4/15
26/26 [==============================] - 1s 52ms/step - loss: 21967898.0000 - mean_absolute_error: 4625.7944
Epoch 5/15
26/26 [==============================] - 1s 54ms/step - loss: 20739262.0000 - mean_absolute_error: 4491.9629
Epoch 6/15
26/26 [==============================] - 1s 55ms/step - loss: 19565278.0000 - mean_absolute_error: 4359.0957
Epoch 7/15
26/26 [==============================] - 1s 54ms/step - loss: 18498986.0000 - mean_absolute_error: 4234.6274
Epoch 8/15
26/26 [==============================] - 1s 52ms/step - loss: 17377182.0000 - mean_absolute_error: 4101.4150
Epoch 9/15
26/26 [=====================

  2%|▏         | 40/2000 [46:11<22:40:40, 41.65s/it]

Epoch 1/15


In [ ]:
data.head()

NameError: name 'data' is not defined

In [ ]:
data_lstm.head

In [ ]:
data_xg.head

In [ ]:
submission = results_df.drop('final_return',axis=1)

In [ ]:
submission.to_csv("submission_stationarity_autoarima.csv")